# Segmenting and Clustering Neighborhoods in Toronto

## 1. Initial Dataframe using Toronto postal code wiki

> Install BeautifulSoup and import Pandas

In [ ]:
!pip install bs4
import pandas as pd
from bs4 import BeautifulSoup
import requests

> Pull data from Wiki and enter into pandas dataframe called 'borough'

In [ ]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(source.content, 'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
borough = pd.DataFrame(df[0])

> Remove rows with Borough not assigned, reset index, and check

In [ ]:
borough = borough[borough.Borough !='Not assigned']
borough = borough.reset_index(drop=True)
borough.head()

> Confirm no neighborhoods with value 'Not assigned'

In [ ]:
NeighNA = borough[(borough['Neighborhood'] == 'Not assigned')]
NeighNA

> Display Number of Rows and Columns

In [ ]:
borough.shape

## 2. Add postal code longitude and latitude values to dataframe

> Read from .csv to create new df with columns Postal Code, Longitude, Latitude

In [ ]:
long_lat = pd.read_csv('https://cocl.us/Geospatial_data')

> Combine df with longitude and latitude values with boroughs df using .merge

In [ ]:
borough11 = pd.merge(borough, long_lat, how='inner')
borough11.reset_index(drop=True, inplace=True)
borough11.head()